In [15]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OrdinalEncoder

In [16]:
X = datasets.fetch_covtype().data[:5000]
y = datasets.fetch_covtype().target[:5000]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

print(X_train.shape, y_train.shape)

enc = OrdinalEncoder()
y_train = enc.fit_transform(y_train.reshape(-1, 1)).reshape(-1, )
y_test = enc.transform(y_test.reshape(-1, 1)).reshape(-1, )
print(np.unique(y_train))  # 7分类任务

X_train, X_test, y_train, y_test = tf.constant(X_train), tf.constant(X_test), tf.constant(y_train), tf.constant(y_test)

(4500, 54) (4500,)
[0. 1. 2. 3. 4. 5. 6.]


In [17]:
inputs = tf.keras.Input(shape=(54,))
x = tf.keras.layers.Dense(256, activation='tanh')(inputs)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dense(128)(x)
predictions = tf.keras.layers.Dense(7)(x)
model = tf.keras.Model(inputs=inputs, outputs=predictions)

model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 54)]              0         
                                                                 
 dense_8 (Dense)             (None, 256)               14080     
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense_9 (Dense)             (None, 128)               32896     
                                                                 
 dense_10 (Dense)            (None, 128)               16512     
                                                                 
 dense_11 (Dense)            (None, 7)                 903       
                                                                 
Total params: 64,391
Trainable params: 64,391
Non-trainable

In [18]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [19]:
callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir='data',
                                   # frequency (in epochs) at which to compute activation and weight histograms for the layers of the model.
                                   # If set to 0, histograms won't be computed.
                                   # Validation data (or split) must be specified for histogram visualizations.
                                   histogram_freq=1,  # 默认histogram_freq=0
                                   # whether to visualize the graph in TensorBoard. The log file can become quite large when write_graph is set to True.
                                   write_graph=True,  # 默认write_graph=True
                                   # whether to write model weights to visualize as image in TensorBoard.
                                   write_images=False,  # 默认write_images=False
                                   # 'batch' or 'epoch' or integer.
                                   # When using 'batch', writes the losses and metrics to TensorBoard after each batch.
                                   # The same applies for 'epoch'.
                                   # If using an integer, let's say 1000, the callback will write the metrics and losses to TensorBoard every 1000 batches.
                                   # Note that writing too frequently to TensorBoard can slow down your training.
                                   update_freq=1,  # 默认update_freq='epoch'
                                   # frequency (in epochs) at which embedding layers will be visualized. If set to 0, embeddings won't be visualized.
                                   embeddings_freq=0  # 默认embeddings_freq=0
                                   )

]
model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test),
          # List of keras.callbacks.Callback instances.
          callbacks=callbacks)

Epoch 1/50
141/141 [==============================] - 1s 5ms/step - loss: 1.3874 - sparse_categorical_accuracy: 0.4482 - val_loss: 1.1467 - val_sparse_categorical_accuracy: 0.5320

In [ ]:
# 终端执行:tensorboard --logdir=data